# eGov APIを利用して法令を抽出する


## 参考文献

https://qiita.com/Lisphilar/items/39ad23ac7ade21313911



In [ ]:
import re
from functools import lru_cache
from pprint import pprint
from xml.etree import ElementTree

# pip install requests
import requests

In [ ]:
@lru_cache
def get_law_dict(category=1):
    # APIから各法令種別に含まれる法令リストを取得
    url = f"https://elaws.e-gov.go.jp/api/1/lawlists/{category}"
    r = requests.get(url)
    # XMLデータの解析
    root = ElementTree.fromstring(r.content.decode(encoding="utf-8"))
    # 辞書{名称: 法令番号}の作成
    names = [e.text for e in root.iter() if e.tag == "LawName"]
    numbers = [e.text for e in root.iter() if e.tag == "LawNo"]
    return {name: num for (name, num) in zip(names, numbers)}

In [ ]:
pprint(get_law_dict(category=2), compact=True)

{'あへん法': '昭和二十九年法律第七十一号',
 'あん摩マツサージ指圧師、はり師、きゆう師等に関する法律': '昭和二十二年法律第二百十七号',
 'いじめ防止対策推進法': '平成二十五年法律第七十一号',
 'お年玉付郵便葉書等に関する法律': '昭和二十四年法律第二百二十四号',
 'お茶の振興に関する法律': '平成二十三年法律第二十一号',
 'がん対策基本法': '平成十八年法律第九十八号',
 'がん登録等の推進に関する法律': '平成二十五年法律第百十一号',
 'こどもの国協会の解散及び事業の承継に関する法律': '昭和五十五年法律第九十一号',
 'こどもの貧困の解消に向けた対策の推進に関する法律': '平成二十五年法律第六十四号',
 'こども基本法': '令和四年法律第七十七号',
 'こども家庭庁設置法': '令和四年法律第七十五号',
 'じん肺法': '昭和三十五年法律第三十号',
 'すき入紙製造取締法': '昭和二十二年法律第百四十九号',
 'たばこ事業法': '昭和五十九年法律第六十八号',
 'たばこ税法': '昭和五十九年法律第七十二号',
 'たばこ耕作組合法': '昭和三十三年法律第百三十五号',
 'とん税法': '昭和三十二年法律第三十七号',
 'と畜場法': '昭和二十八年法律第百十四号',
 'へき地教育振興法': '昭和二十九年法律第百四十三号',
 'まぐろ資源の保存及び管理の強化に関する特別措置法': '平成八年法律第百一号',
 'まち・ひと・しごと創生法': '平成二十六年法律第百三十六号',
 'ものづくり基盤技術振興基本法': '平成十一年法律第二号',
 'アイヌの人々の誇りが尊重される社会を実現するための施策の推進に関する法律': '平成三十一年法律第十六号',
 'アジア開発銀行への加盟に伴う措置に関する法律': '昭和四十一年法律第百三十八号',
 'アフリカ開発基金への参加に伴う措置に関する法律': '昭和四十八年法律第三十八号',
 'アフリカ開発銀行への加盟に伴う措置に関する法律': '昭和五十六年法律第四十一号',
 'アルコール事業法': '平成十二年法律第三十六号',
 'アルコール健康障害対策基本法': '平成二十五年法律第百九号',
 'アレルギー疾患対策基本法': 

In [ ]:
def get_law_number(keyword, category=1):
    """
    Return the law number.
    This will be retrieved from e-Gov (https://www.e-gov.go.jp/)

    Args:
        keyword (str): keyword of the law name
        category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

    Returns:
        dict(str, str): dictionary of law name (key) and law number (value)
    """
    law_dict = get_law_dict(category=category)
    return {k: v for (k, v) in law_dict.items() if keyword in k}

In [ ]:
print(get_law_number("医薬品の臨床試験", category=4))

{'医薬品の臨床試験の実施の基準に関する省令': '平成九年厚生省令第二十八号', '動物用医薬品の臨床試験の実施の基準に関する省令': '平成九年農林水産省令第七十五号'}


In [ ]:
@lru_cache
def get_raw(number):
    """
    Retrieve contents of the law specified with law number from e-Gov API.

    Args:
        number (str): Number of the law, like '平成九年厚生省令第二十八号'

    Returns:
        raw (list[str]): raw contents of J-GCP
    """
    url = f"https://elaws.e-gov.go.jp/api/1/lawdata/{number}"
    r = requests.get(url)
    root = ElementTree.fromstring(r.content.decode(encoding="utf-8"))
    contents = [e.text.strip() for e in root.iter() if e.text]
    return [t for t in contents if t]

In [ ]:
gcp_raw = get_raw("平成九年厚生省令第二十八号")
pprint(gcp_raw, compact=False)

['0',
 '平成九年厚生省令第二十八号',
 '平成九年厚生省令第二十八号',
 '医薬品の臨床試験の実施の基準に関する省令',
 '薬事法（昭和三十五年法律第百四十五号）第十四条第三項（同条第六項、同法第十九条の二第四項及び第二十三条において準用する場合を含む。）、第十四条の四第四項並びに第十四条の五第四項（これらの規定を同法第十九条の四及び第二十三条において準用する場合を含む。）、第八十条の二第一項、第四項及び第五項並びに第八十二条の規定に基づき、医薬品の臨床試験の実施の基準に関する省令を次のように定める。',
 '目次',
 '第一章\u3000総則',
 '（第一条―第三条）',
 '第二章\u3000治験の準備に関する基準',
 '第一節\u3000治験の依頼をしようとする者による治験の準備に関する基準',
 '（第四条―第十五条）',
 '第二節\u3000自ら治験を実施しようとする者による治験の準備に関する基準',
 '（第十五条の二―第十五条の九）',
 '第三章\u3000治験の管理に関する基準',
 '第一節\u3000治験依頼者による治験の管理に関する基準',
 '（第十六条―第二十六条）',
 '第二節\u3000自ら治験を実施する者による治験の管理に関する基準',
 '（第二十六条の二―第二十六条の十二）',
 '第四章\u3000治験を行う基準',
 '（第二十七条―第五十五条）',
 '第一節\u3000治験審査委員会',
 '（第二十七条―第三十四条）',
 '第二節\u3000実施医療機関',
 '（第三十五条―第四十一条）',
 '第三節\u3000治験責任医師',
 '（第四十二条―第四十九条）',
 '第四節\u3000被験者の同意',
 '（第五十条―第五十五条）',
 '第五章\u3000再審査等の資料の基準',
 '（第五十六条）',
 '第六章\u3000治験の依頼等の基準',
 '（第五十七条―第五十九条）',
 '附則',
 '第一章\u3000総則',
 '（趣旨）',
 '第一条',
 'この省令は、被験者の人権の保護、安全の保持及び福祉の向上を図り、治験の科学的な質及び成績の信頼性を確保するため、医薬品、医療機器等の品質、有効性及び安全性の確保等に関する法律（昭和三十五年法律第百四十五号。以下「法」という

In [ ]:
def preprocess_gcp(raw):
    """
    Perform pre-processing on raw contents of J-GCP.

    Args:
        raw (list[str]): raw contents of J-GCP

    Returns:
        str: pre-processed string of J-GCP

    Notes:
        - Article 56 will be removed.
        - Strings enclosed with （ and ） will be removed.
        - 「 and 」 will be removed.
    """
    # contents = raw[:]
    # Remove article 56
    contents = raw[: raw.index("第五十六条")]
    # Select sentenses
    contents = [s for s in contents if s.endswith("。")]
    # Join the sentenses
    gcp = "".join(contents)
    # 「 and 」 will be removed
    gcp = gcp.translate(str.maketrans({"「": "", "」": ""}))
    # 　Strings enclosed with （ and ） will be removed
    return re.sub("（[^（|^）]*）", "", gcp)

In [ ]:
gcp = preprocess_gcp(gcp_raw)
gcp

'薬事法第十四条第三項、第十四条の四第四項並びに第十四条の五第四項、第八十条の二第一項、第四項及び第五項並びに第八十二条の規定に基づき、医薬品の臨床試験の実施の基準に関する省令を次のように定める。この省令は、被験者の人権の保護、安全の保持及び福祉の向上を図り、治験の科学的な質及び成績の信頼性を確保するため、医薬品、医療機器等の品質、有効性及び安全性の確保等に関する法律第十四条第三項及び第十二項（同条第十五項及び法第十九条の二第五項において準用する場合並びに法第十四条の二の二第五項において読み替えて適用する場合を含む。以下同じ。）並びに法第十四条の四第五項及び第十四条の六第四項の厚生労働省令で定める基準のうち医薬品の臨床試験の実施に係るもの並びに法第八十条の二第一項、第四項及び第五項に規定する厚生労働省令で定める基準を定めるものとする。この省令において製造販売後臨床試験とは、医薬品の製造販売後の調査及び試験の実施の基準に関する省令第二条第一項第三号に規定する製造販売後臨床試験をいう。この省令において実施医療機関とは、治験又は製造販売後臨床試験を行う医療機関をいう。この省令において治験責任医師とは、実施医療機関において治験に係る業務を統括する医師又は歯科医師をいう。この省令において製造販売後臨床試験責任医師とは、実施医療機関において製造販売後臨床試験に係る業務を統括する医師又は歯科医師をいう。この省令において被験薬とは、治験の対象とされる薬物又は製造販売後臨床試験の対象とされる医薬品をいう。この省令において対照薬とは、治験又は製造販売後臨床試験において被験薬と比較する目的で用いられる薬物をいう。この省令において治験薬とは、被験薬及び対照薬をいう。この省令において製造販売後臨床試験薬とは、被験薬及び対照薬をいう。この省令において治験使用薬とは、被験薬並びに被験薬の有効性及び安全性の評価のために使用する薬物をいう。この省令において治験使用薬等とは、治験使用薬又は治験使用薬と成分が同一性を有すると認められる薬物をいう。この省令において製造販売後臨床試験使用薬とは、被験薬並びに被験薬の有効性及び安全性の評価のために使用する薬物をいう。この省令において製造販売後臨床試験使用薬等とは、製造販売後臨床試験使用薬又は製造販売後臨床試験使用薬と成分が同一性を有すると認められる薬物をいう。

In [13]:
class LawLoader(object):
    """
    Prepare law data with e-Gov (https://www.e-gov.go.jp/) site.

    Args:
        category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)
    """

    def __init__(self, category=1):
        self.law_dict = self._get_law_dict(category=category)
        self.content_dict = {}

    @staticmethod
    def _get_xml(url):
        """
        Get XML data from e-Gov API.

        Args:
            url (str): key of the API

        Returns:
            xml.ElementTree: element tree of the XML data
        """
        r = requests.get(url)
        return ElementTree.fromstring(r.content.decode(encoding="utf-8"))

    def _get_law_dict(self, category):
        """
        Return dictionary of law names and numbers.

        Args:
            category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

        Returns:
            dict(str, str): dictionary of law names (keys) and numbers (values)
        """
        url = f"https://elaws.e-gov.go.jp/api/1/lawlists/{category}"
        root = self._get_xml(url)
        names = [e.text for e in root.iter() if e.tag == "LawName"]
        numbers = [e.text for e in root.iter() if e.tag == "LawNo"]
        return {name: num for (name, num) in zip(names, numbers)}

    def get_law_number(self, keyword, category=1):
        """
        Return the law number.
        This will be retrieved from e-Gov (https://www.e-gov.go.jp/)

        Args:
            keyword (str): keyword of the law name
            category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

        Returns:
            dict(str, str): dictionary of law name (key) and law number (value)
        """
        return {k: v for (k, v) in self.law_dict.items() if keyword in k}

    def get_raw(self, number):
        """
        Args:
            number (str): Number of the law, like '平成九年厚生省令第二十八号'

        Returns:
            raw (list[str]): raw contents of J-GCP
        """
        if number in self.content_dict:
            return self.content_dict[number]
        url = f"https://elaws.e-gov.go.jp/api/1/lawdata/{number}"
        root = self._get_xml(url)
        contents = [e.text.strip() for e in root.iter() if e.text]
        raw = [t for t in contents if t]
        self.content_dict = {number: raw}
        return raw

    @staticmethod
    def pre_process(raw):
        """
        Perform pre-processing on raw contents.

        Args:
            raw (list[str]): raw contents

        Returns:
            str: pre-processed string

        Notes:
            - Strings enclosed with （ and ） will be removed.
            - 「 and 」 will be removed.
        """
        contents = [s for s in raw if s.endswith("。")]
        string = "".join(contents)
        string = string.translate(str.maketrans({"「": "", "」": ""}))
        return re.sub("（[^（|^）]*）", "", string)

    def gcp(self):
        """
        Perform pre-processing on raw contents of J-GCP.

        Args:
            raw (list[str]): raw contents of J-GCP

        Returns:
            str: pre-processed string of J-GCP

        Notes:
            - Article 56 will be removed.
            - Strings enclosed with （ and ） will be removed.
            - 「 and 」 will be removed.
        """
        number_dict = self.get_law_number("医薬品の臨床試験")
        number = number_dict["医薬品の臨床試験の実施の基準に関する省令"]
        raw = self.get_raw(number)
        raw_without56 = raw[: raw.index("第五十六条")]
        return self.pre_process(raw_without56)

In [ ]:
# The Constitution of Japan
loader2 = LawLoader(category=2)
consti_number = loader2.get_law_number("日本国憲法")
print(consti_number)  # -> '昭和二十一年憲法'
consti_raw = loader2.get_raw("昭和二十一年憲法")
consti = loader2.pre_process(consti_raw)
# J-GCP：データ整形を含めてメソッドとして登録済
loader4 = LawLoader(category=4)
gcp = loader4.gcp()

{'日本国憲法': '昭和二十一年憲法', '昭和二十二年法律第七十二号（日本国憲法施行の際現に効力を有する命令の規定の効力等に関する法律）': '昭和二十二年法律第七十二号', '日本国憲法の改正手続に関する法律': '平成十九年法律第五十一号'}
